# Environmental and Particulate Matter Data Analysis

This notebook analyzes environmental sensor data including temperature, humidity, and particulate matter measurements.

## 1. Data Loading and Initial Exploration

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
import plotly.express as px
from datetime import datetime, timedelta
import geopandas as gpd
from shapely.geometry import Point


In [3]:
file_path = 'DATA.TXT'  # Update this path if needed
data = pd.read_csv(file_path, sep=';')

In [4]:
data['geometry'] = data.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf = gpd.GeoDataFrame(data, geometry='geometry', crs='EPSG:4326')
gdf.to_file('sensebox_data.geojson', driver='GeoJSON')


In [5]:
# print min and max values of Temperature , Humidity,  PM1_0  PM2_5  PM4_0  PM10_0  
print("Min and Max values of Temperature, Humidity, PM1_0, PM2_5, PM4_0, PM10_0:")
print("Temperature: Min =", data['Temperature'].min(), ", Max =", data['Temperature'].max())
print("Humidity: Min =", data['Humidity'].min(), ", Max =", data['Humidity'].max())
print("PM1_0: Min =", data['PM1_0'].min(), ", Max =", data['PM1_0'].max())
print("PM2_5: Min =", data['PM2_5'].min(), ", Max =", data['PM2_5'].max())
print("PM4_0: Min =", data['PM4_0'].min(), ", Max =", data['PM4_0'].max())
print("PM10_0: Min =", data['PM10_0'].min(), ", Max =", data['PM10_0'].max())


Min and Max values of Temperature, Humidity, PM1_0, PM2_5, PM4_0, PM10_0:
Temperature: Min = 23.23 , Max = 44.58
Humidity: Min = 20.32 , Max = 45.5
PM1_0: Min = 2.78 , Max = 20.27
PM2_5: Min = 2.94 , Max = 21.43
PM4_0: Min = 2.94 , Max = 21.43
PM10_0: Min = 2.94 , Max = 21.43


In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import json

csv_meteo = 'ugz_ogd_meteo_h1_2025.csv'
csv_air = 'ugz_ogd_air_h1_2025.csv'
json_path = 'uzg_ogd_metadaten.json'
selected_date = '2025-05-02'

# === LOAD METEO DATA ===
meteo = pd.read_csv(csv_meteo)
meteo['Datum'] = pd.to_datetime(meteo['Datum'])
meteo_filtered = meteo[meteo['Datum'].dt.date == pd.to_datetime(selected_date).date()].copy()
meteo_filtered['Hour'] = meteo_filtered['Datum'].dt.hour

# === LOAD AIR QUALITY DATA ===
air = pd.read_csv(csv_air)
air['Datum'] = pd.to_datetime(air['Datum'])
air_filtered = air[air['Datum'].dt.date == pd.to_datetime(selected_date).date()].copy()
air_filtered['Hour'] = air_filtered['Datum'].dt.hour

# === COMBINE METEO + AIR ===
combined = pd.concat([meteo_filtered, air_filtered], ignore_index=True)

# === AGGREGATE HOURLY AVERAGES ===
hourly_avg = (combined
              .groupby(['Standort', 'Parameter', 'Hour'])['Wert']
              .mean()
              .reset_index()
              .pivot(index=['Standort', 'Hour'], columns='Parameter', values='Wert')
              .reset_index())

# === LOAD STATION METADATA ===
with open(json_path, 'r', encoding='utf-8') as f:
    stations = json.load(f)

stations_df = pd.DataFrame(stations['Standorte']).rename(columns={
    'Koordinaten_WGS84_lat': 'Lat',
    'Koordinaten_WGS84_lng': 'Lng',
    'Höhe [M.ü.M.]': 'Höhe'
})

# === MERGE WITH COORDINATES ===
gdf = hourly_avg.merge(stations_df, left_on='Standort', right_on='ID', how='left')
gdf['geometry'] = gdf.apply(lambda row: Point(row['Lng'], row['Lat']), axis=1)
gdf = gpd.GeoDataFrame(gdf, geometry='geometry', crs='EPSG:4326')

In [7]:
columns_to_keep = [
    'Standort', 'Hour', 'T', 'Hr',  # Temperature and Humidity
    'PM1', 'PM2.5', 'PM4', 'PM10',  # PM values (match based on actual names)
    'Lat', 'Lng', 'Name', 'geometry'
]
gdf_filtered = gdf[[col for col in columns_to_keep if col in gdf.columns]]


In [8]:
print(gdf_filtered.head())
gdf_filtered.to_file('zh_confirmation_data.geojson', driver='GeoJSON')

           Standort  Hour      T     Hr  PM2.5   PM10      Lat     Lng  \
0  Zch_Heubeeribüel     0  17.00  44.37   6.20   9.89  47.3815  8.5659   
1  Zch_Heubeeribüel     1  15.89  49.64   6.10  10.11  47.3815  8.5659   
2  Zch_Heubeeribüel     2  15.07  56.14   6.44  10.71  47.3815  8.5659   
3  Zch_Heubeeribüel     3  14.58  59.47   6.44  10.57  47.3815  8.5659   
4  Zch_Heubeeribüel     4  14.37  60.49   6.42  10.39  47.3815  8.5659   

                  Name                  geometry  
0  Zürich Heubeeribüel  POINT (8.56590 47.38150)  
1  Zürich Heubeeribüel  POINT (8.56590 47.38150)  
2  Zürich Heubeeribüel  POINT (8.56590 47.38150)  
3  Zürich Heubeeribüel  POINT (8.56590 47.38150)  
4  Zürich Heubeeribüel  POINT (8.56590 47.38150)  


In [9]:
# Rename columns to match sensebox dataset
gdf_filtered = gdf_filtered.rename(columns={
    "T": "Temperature",
    "Hr": "Humidity",
    "PM2.5": "PM2_5",
    "PM10": "PM10_0"
})
gdf_filtered.to_file("zh_confirmation_data.geojson", driver="GeoJSON")

In [ ]:
sensebox = gpd.read_file("sensebox_data.geojson")
reference = gpd.read_file("zh_confirmation_data.geojson")



In [14]:
# get crs of both dataframes
print("CRS of sensebox data:", sensebox.crs)
print("CRS of reference data:", reference.crs)

CRS of sensebox data: EPSG:4326
CRS of reference data: EPSG:4326


In [11]:
def find_closest_reference(sensebox_row, ref_gdf):
    nearest_idx = ref_gdf.geometry.distance(sensebox_row.geometry).idxmin()
    nearest_row = ref_gdf.loc[nearest_idx]
    return pd.Series({
        "ref_Temperature": nearest_row.get("Temperature"),
        "ref_Humidity": nearest_row.get("Humidity"),
        "ref_PM1_0": nearest_row.get("PM1", None),  # use .get() to be safe
        "ref_PM2_5": nearest_row.get("PM2_5", None),
        "ref_PM4_0": nearest_row.get("PM4", None),
        "ref_PM10_0": nearest_row.get("PM10_0", None),
    })


In [12]:
nearest_info = sensebox.apply(find_closest_reference, axis=1, ref_gdf=reference)
sensebox = pd.concat([sensebox, nearest_info], axis=1)


C:\Users\Noe\AppData\Local\Temp\ipykernel_24392\1325897213.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest_idx = ref_gdf.geometry.distance(sensebox_row.geometry).idxmin()
C:\Users\Noe\AppData\Local\Temp\ipykernel_24392\1325897213.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest_idx = ref_gdf.geometry.distance(sensebox_row.geometry).idxmin()
C:\Users\Noe\AppData\Local\Temp\ipykernel_24392\1325897213.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest_idx = ref_gdf.geometry.distance(sensebox_row.geometry).idxmin()
C:\Users\Noe\AppData\Local\T

In [15]:
for var in ["Temperature", "Humidity", "PM1_0", "PM2_5", "PM4_0", "PM10_0"]:
    sensebox[f"diff_{var}"] = sensebox[var] - sensebox[f"ref_{var}"]


In [16]:
sensebox.to_file("sensebox_with_differences.geojson", driver="GeoJSON")
print("✅ Done! You can now use sensebox_with_differences.geojson in your web app.")


✅ Done! You can now use sensebox_with_differences.geojson in your web app.


________

In [2]:
import pandas as pd
import json

# Load GeoJSON file
geojson_path = "sensebox_with_differences.geojson"
csv_output_path = "sensebox_with_differences.csv"

with open(geojson_path, "r", encoding="utf-8") as f:
    geojson_data = json.load(f)

# Extract features and flatten properties + geometry
records = []
for feature in geojson_data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    props["Longitude"] = coords[0]
    props["Latitude"] = coords[1]
    records.append(props)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(records)
df.to_csv(csv_output_path, index=False)

csv_output_path


'sensebox_with_differences.csv'

In [3]:
import json

# Path to your input and output files
input_file = "sensebox_with_differences.geojson"
output_file = "sensebox_with_differences_FIXED.geojson"

# List of all expected fields
required_fields = [
    "ref_Temperature", "ref_Humidity", "ref_PM1_0", "ref_PM2_5", "ref_PM4_0", "ref_PM10_0",
    "diff_Temperature", "diff_Humidity", "diff_PM1_0", "diff_PM2_5", "diff_PM4_0", "diff_PM10_0"
]

# Load the GeoJSON
with open(input_file, "r", encoding="utf-8") as f:
    geojson = json.load(f)

# Ensure all required fields exist in every feature
for feature in geojson["features"]:
    props = feature.get("properties", {})
    for field in required_fields:
        if field not in props:
            props[field] = None  # add missing field with null value
    feature["properties"] = props  # reassign to be safe

# Save the fixed GeoJSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(geojson, f, indent=2)

print("✅ GeoJSON fixed and saved as", output_file)


✅ GeoJSON fixed and saved as sensebox_with_differences_FIXED.geojson


In [4]:
import json

# Load the original file
input_file = "sensebox_with_differences.geojson"
output_file = "sensebox_with_differences_FIXED_DUMMY.geojson"

# Fields we want to preserve
fields = [
    "ref_Temperature", "ref_Humidity", "ref_PM1_0", "ref_PM2_5", "ref_PM4_0", "ref_PM10_0",
    "diff_Temperature", "diff_Humidity", "diff_PM1_0", "diff_PM2_5", "diff_PM4_0", "diff_PM10_0"
]

# Load geojson
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Fill all fields with 0.0 in the first feature
first_props = data["features"][0]["properties"]
for field in fields:
    first_props[field] = 0.0  # Dummy numeric value

# Ensure all other features at least have those fields (set to null if missing)
for feat in data["features"][1:]:
    props = feat["properties"]
    for field in fields:
        if field not in props:
            props[field] = None

# Save the result
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)

print("✅ Fixed file saved:", output_file)


✅ Fixed file saved: sensebox_with_differences_FIXED_DUMMY.geojson


In [5]:
import pandas as pd

# Load the original CSV
csv_path = "sensebox_with_differences.csv"
output_path = "sensebox_with_differences_FILLED.csv"

# Read CSV
df = pd.read_csv(csv_path)

# Replace all NaN values with -9999
df_filled = df.fillna(-9999)

# Save the updated CSV
df_filled.to_csv(output_path, index=False)

output_path


'sensebox_with_differences_FILLED.csv'